<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#notebook-setup" data-toc-modified-id="notebook-setup-0.1">notebook setup</a></span></li></ul></li><li><span><a href="#Prelminary-data-analysis" data-toc-modified-id="Prelminary-data-analysis-1">Prelminary data analysis</a></span><ul class="toc-item"><li><span><a href="#Visualize-radon-levels" data-toc-modified-id="Visualize-radon-levels-1.1">Visualize radon levels</a></span></li><li><span><a href="#Relationship-between-radon-and-uranium-levels" data-toc-modified-id="Relationship-between-radon-and-uranium-levels-1.2">Relationship between radon and uranium levels</a></span></li></ul></li><li><span><a href="#Bayesian-modeling-with-CmdStanPy" data-toc-modified-id="Bayesian-modeling-with-CmdStanPy-2">Bayesian modeling with CmdStanPy</a></span><ul class="toc-item"><li><span><a href="#Model-Comparison" data-toc-modified-id="Model-Comparison-2.1">Model Comparison</a></span></li><li><span><a href="#Compare-estimates-of-parameter-beta" data-toc-modified-id="Compare-estimates-of-parameter-beta-2.2">Compare estimates of parameter <code>beta</code></a></span></li></ul></li><li><span><a href="#Posterior-Predictive-Checking" data-toc-modified-id="Posterior-Predictive-Checking-3">Posterior Predictive Checking</a></span><ul class="toc-item"><li><span><a href="#Posterior-Predictive-Test---Partial-Pooling-Model" data-toc-modified-id="Posterior-Predictive-Test---Partial-Pooling-Model-3.1">Posterior Predictive Test - Partial Pooling Model</a></span></li><li><span><a href="#Posterior-Predictive-Density-Plot---Partial-Pooling-Model" data-toc-modified-id="Posterior-Predictive-Density-Plot---Partial-Pooling-Model-3.2">Posterior Predictive Density Plot - Partial Pooling Model</a></span></li></ul></li><li><span><a href="#Concluding-remarks" data-toc-modified-id="Concluding-remarks-4">Concluding remarks</a></span></li><li><span><a href="#References-and-Resources" data-toc-modified-id="References-and-Resources-5">References and Resources</a></span></li></ul></div>

# Probabalistic Programming with CmdStanPy and plotnine

### notebook setup

In [ ]:
# import all libraries used in this notebook
import os
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel

# plotting libs
import matplotlib.pyplot as plt
import plotnine as p9

# suppress plotnine warnings
import warnings
warnings.filterwarnings('ignore')

# setup plotnine look and feel
p9.theme_set(
  p9.theme_grey() + 
  p9.theme(text=p9.element_text(size=10),
        plot_title=p9.element_text(size=12),
        axis_title_x=p9.element_text(size=11),
        axis_title_y=p9.element_text(size=11),
        axis_text_x=p9.element_text(size=8),
        axis_text_y=p9.element_text(size=8)
       )
)
xlabels_90 = p9.theme(axis_text_x = p9.element_text(angle=90, hjust=1))

pd.set_option('display.precision', 2)

import logging
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

## Prelminary data analysis

Best practice: put data on the log scale

In [ ]:
mn_radon = pd.read_csv(os.path.join('data','mn_radon.csv'))
mn_counties = pd.read_csv(os.path.join('data','mn_uranium.csv'))

In [ ]:
mn_radon.iloc[[1,5,58]]

In [ ]:
mn_counties[:3]

### Visualize radon levels

* Use [plotnine.stat_density](https://plotnine.readthedocs.io/en/stable/generated/plotnine.stats.stat_density.html) to compute the density of column `log_radon`.
* Map column `floor` to aesthetic 'color'


In [ ]:
(p9.ggplot(data=mn_radon, mapping=p9.aes(x='log_radon', color='factor(floor)'))
    + p9.stat_density(geom='line')
    + p9.scale_color_manual(['darkorange','purple'], name='floor')
    + p9.xlab("log radon levels")
    + p9.theme(figure_size=(6,4))
)

In [ ]:
(p9.ggplot(data=mn_radon, mapping=p9.aes(x='log_radon', color='factor(floor)', fill='factor(floor)'))
    + p9.geom_histogram(alpha=0.7, binwidth=0.2)
    + p9.scale_color_manual(['darkgreen','darkblue'])
    + p9.scale_fill_manual(['orange','violet'])
    + p9.theme(figure_size=(6,4))
)

The raw counts histogram reveals that most of the observations in the survey were taken on the basement level.  Let's compute the exact percentages.

In [ ]:
pct_1 = round((mn_radon.floor.sum() / len(mn_radon) * 100))
pct_0= round(100 - pct_1)
print(f'floor 0: {pct_0}%\nfloor 1: {pct_1}%')

How many counties have data from both floors?


In [ ]:
print(f'Number of counties: {mn_radon.county.nunique()}')
print(f'Counties with measurements from floor 0: {mn_radon[mn_radon["floor"]==0]["county"].nunique()}')
print(f'Counties with measurements from floor 1: {mn_radon[mn_radon["floor"]==1]["county"].nunique()}')

*Note:  2 counties with no data!*

### Relationship between radon and uranium levels

In [ ]:
mn_radon[["radon", "uranium", "log_radon", "log_uranium"]].describe()[1:]

Compare the background soil uranium (y-axis) to the number of measurements per county (x-axis).
Add text labels to counties with more than 25 observations.

In [ ]:
(p9.ggplot(data=mn_counties, mapping=p9.aes(x='homes', y='log_uranium'))
    + p9.geom_point(fill='orange', color='darkred')
    + p9.geom_text(data=mn_counties[mn_counties['homes']>25],
                   mapping=p9.aes(label='county'),
                   size=8, nudge_x=4, nudge_y=0.1)
    + p9.xlab("homes per county") + p9.ylab("county soil log_uranium")
    + p9.theme(figure_size=(12,6))
)

Show the range of measurements per county ordered by soil uranium, via
[plotnine.geom_boxplot](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_boxplot.html#plotnine.geoms.geom_boxplot)

In [ ]:
uranium_asc = mn_counties.sort_values(by='log_uranium').reset_index()

(p9.ggplot(data=mn_radon, mapping=p9.aes(x='county',y='log_radon'))
    + p9.geom_boxplot(width=2, varwidth=True, outlier_alpha=0.4)
    + p9.scale_x_discrete(limits=uranium_asc['county'], expand=(0,1))
    + p9.ggtitle("Counties ordered by soil uranium low (left) to high (right)")
    + p9.ylab("range of home radon measurements")
    + xlabels_90
    + p9.theme(figure_size=(12,6))
)

## Bayesian modeling with CmdStanPy

In [ ]:
# assemble data
radon_data = {"N": len(mn_radon), "x": mn_radon.floor.astype(float), "y": mn_radon.log_radon}
radon_data["J"] = 85
radon_data["county"] = mn_radon.county_id

In [ ]:
# compile, fit models

complete_pooling_model = CmdStanModel(stan_file=os.path.join('stan', 'radon_cp.stan'))
complete_pooling_fit = complete_pooling_model.sample(data=radon_data)

no_pooling_model = CmdStanModel(stan_file=os.path.join('stan', 'radon_np.stan'))
no_pooling_fit = no_pooling_model.sample(data=radon_data)

partial_pooling_alpha_model = CmdStanModel(stan_file=os.path.join('stan', 'radon_pp_alpha.stan'))
partial_pooling_alpha_fit = partial_pooling_alpha_model.sample(data=radon_data)

Access variables of interest from CmdStanMCMC fit objects.

In [ ]:
complete_pool_alpha_mean = np.mean(complete_pooling_fit.stan_variable('alpha'))

no_pool_alpha = no_pooling_fit.stan_variable('alpha')
no_pool_alpha_mean = np.mean(no_pool_alpha, axis=0)  # axis=0 uses all rows, i.e., per-column mean
no_pool_alpha_lower = np.quantile(no_pool_alpha, 0.16, axis=0)
no_pool_alpha_upper = np.quantile(no_pool_alpha, 0.84, axis=0)
no_pool_alpha_pd = pd.DataFrame(data={
    "mean": no_pool_alpha_mean,
    "upper": no_pool_alpha_upper, 
    "lower": no_pool_alpha_lower, 
    "county":mn_counties['county']
})

part_pool_mu_alpha = partial_pooling_alpha_fit.stan_variable('mu_alpha').mean()
part_pool_alpha = partial_pooling_alpha_fit.stan_variable('alpha')
part_pool_alpha_mean = np.mean(part_pool_alpha, axis=0)
part_pool_alpha_lower = np.quantile(part_pool_alpha, 0.16, axis=0)
part_pool_alpha_upper = np.quantile(part_pool_alpha, 0.84, axis=0)
part_pool_alpha_pd = pd.DataFrame(
    data={
        "mean": part_pool_alpha_mean,
        "upper": part_pool_alpha_upper, 
        "lower": part_pool_alpha_lower, 
        "county":mn_counties['county']
    }
)

### Model Comparison

* green - global intercept, complete-pooling model
* orange - per-county intercepts, no-pooling model
* blue - estimate from multilevel model; partial-pooling

In [ ]:
pop_desc = mn_counties.sort_values(by='homes', ascending=False).reset_index()

p_compare = (p9.ggplot(data=part_pool_alpha_pd)
 + p9.geom_segment(
     mapping=p9.aes(x='county', xend='county', y='lower', yend='upper'),
     size=1.7, color='blue', alpha=0.7,
 )
 + p9.geom_segment(data=no_pool_alpha_pd,
     mapping=p9.aes(x='county', xend='county', y='lower', yend='upper'),
     size=1.4, color='orange', alpha=0.6,
 )
 + p9.geom_point(mapping=p9.aes(x='county', y='mean'))
 + p9.geom_hline(yintercept=part_pool_mu_alpha, color='darkblue', size=1)
 + p9.geom_hline(yintercept=complete_pool_alpha_mean, color='green', size=0.5)
 + p9.scale_x_discrete(limits=pop_desc['county']) + p9.scales.ylim(0,3.5)
 + p9.ggtitle("multilevel varying intercept model estimates for alpha (basement log_radon level)")
 + p9.xlab("ordered by observations per county, desc") + p9.ylab("central 67% interval")
 + xlabels_90
)
p_compare + p9.theme(figure_size=(12,6)) 

### Compare estimates of parameter `beta`

In [ ]:
df_beta = pd.DataFrame({
      'complete_pool_beta': complete_pooling_fit.stan_variable('beta'),
      'no_pool_beta' : no_pooling_fit.stan_variable('beta'),
      'partial_pool_beta' : partial_pooling_alpha_fit.stan_variable('beta')
      })
df_beta.describe()[1:]

In [ ]:
(p9.ggplot(data=df_beta.melt(), mapping=p9.aes(x='value', color='variable'))
    + p9.stat_density(geom='line')
    + p9.scale_color_manual(['green','orange','blue'], name='model')
    + p9.ggtitle("Estimates of slope parameter beta")
    + p9.xlab('')
    + p9.theme(figure_size=(7,5))
)

## Posterior Predictive Checking

"Posterior predictive checks are the unit tests of probabilistic programming." -- Ben Goodrich


Simulate a new data set using the fitted model parameters:

```
generated quantities {
  array[N] real y_rep =
      normal_rng(alpha[county] + beta * x, sigma);
}
```

* Each iteration of the sampler generates a new dataset `y_rep` (replicate)

* Compute statistics on dataset `y_rep`; compare with those on `y`.

### Posterior Predictive Test - Partial Pooling Model

Estimated mean home radon level per county overlaid on boxplot of `y`

In [ ]:
y_rep_pp = partial_pooling_alpha_fit.draws_pd(vars='y_rep')

# for each of the 85 counties, estimate the median y_rep
stat_median = []
for i in range(1,86):
    idxs = mn_radon.index[mn_radon['county_id'] == i].tolist()
    stat_median.append(np.median(y_rep_pp.iloc[:, idxs].to_numpy().flatten()))

(p9.ggplot()
  + p9.geom_boxplot(data=mn_radon,
                    mapping=p9.aes(x='county',y='log_radon'), color='orange', fatten=2, alpha=0.7, outlier_alpha=0.3)
  + p9.geom_point(mapping=p9.aes(x=mn_counties.county, y=stat_median), color='purple')
  + xlabels_90
  + p9.theme(figure_size=(12,3))
)

### Posterior Predictive Density Plot - Partial Pooling Model

Overlay the density plots of a random sample of `y_rep` with density plot of `y`

In [ ]:
# get a random sample of the draws
sz = 100
y_rep_pp = partial_pooling_alpha_fit.draws_pd(vars='y_rep')

# each column is a replicate of the data, using estimates of alpha, beta
y_rep_pp_sample = y_rep_pp.sample(sz).reset_index(drop=True).T

# plot actual distribution of the data against predicted new data
pp_ppc = p9.ggplot()
for i in range(sz):
    pp_ppc = pp_ppc + p9.stat_density(mapping=p9.aes(x=y_rep_pp_sample[i]), geom='line', color='lightblue', alpha=0.4)
pp_ppc = (pp_ppc 
          + p9.stat_density(data=mn_radon, mapping=p9.aes(x='log_radon'), geom='line', color='darkblue', size=1.1)
          + p9.xlab("log radon levels") + p9.ylab("density")
          + p9.theme(figure_size=(6,4))
         )
pp_ppc

## Concluding remarks

Data visualization drives design, testing, and documentation.

Prior and posterior predictive checks validate the model specification.

Plotnine provides a rich set of visualizations to demonstrate and communicate
data, inferences, and predictions.

We welcome feedback and ideas for ways to turn these plots into easy-to-use tools.


## References and Resources

[Stan Tutorials YouTube Playlist](https://www.youtube.com/playlist?list=PLCrWEzJgSUqwL85xIj1wubGdY15C5Gf7H) Maggie Lieu - a series of introductory videos on Bayesian modeling with Stan

[Stan User's Guide](https://mc-stan.org/docs/stan-users-guide/index.html)

[Visualization in Bayesian workflow](https://rss.onlinelibrary.wiley.com/doi/pdfdirect/10.1111/rssa.12378)
Jonah Gabry, Daniel Simpson, Aki Vehtari, Michael Betancourt, and Andrew Gelman, 2019

[Statistical rethinking](https://xcelab.net/rm/statistical-rethinking/) by Richard McElreath -
an intro-stats/linear models course taught from a Bayesian perspective.

 - [GitHub page](https://github.com/rmcelreath/rethinking)
 - [Course page](https://www.youtube.com/playlist?list=PLDcUM9US4XdMROZ57-OIRtIK0aOynbgZN)

[Making Plots With plotnine](https://datacarpentry.org/python-ecology-lesson/07-visualization-ggplot-python/) - plotnine tutorial notebook.

[CmdStanPy Documentation](https://mc-stan.org/cmdstanpy/index.html)
